# **Práctica 8: Clasificación con Validación Cruzada**

En este proyecto se va a aplicar tres modelos de Clasificación :
- Árboles de Decisión
- Regresión Logística
- K-Nearest Neighbors
  
Para predecir si un tumor es maligno o benigno.


Se utilizará validación cruzada para evaluar el rendimiento de cada modelo con diferentes valores de k (k=5 y k=10).

## **Importamos Librerias**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## **Cargamos los Datos**
Hacemos un análisis usando las estadisticas básicas y verificamos la estructura de los datos (numero de muestras, caracteristicas, etc)

El dataset que tenemos sobre cancer tiene varias características y una columna que indica si el tumor es benigno o maligno.

In [ ]:
# Carga del dataset y exploracion
cancer = pd.read_csv("C:data.csv")
print(cancer)
print(cancer.info())

In [ ]:
# análisis estadístico con media, mediana, minimo, maximo. 
# rango intercuartílico (que nos ayuda a cuantificar una muestra eliminando los valores extremadamente alejados) 
# para valores atípicos y distribución de los datos 
print (cancer.describe().T)

In [ ]:
# vamos a hacer dos cambios
# la columna 32 "Unnamed: 32" son todo valores nulos y vamos a eliminarla
# vamos a detectar datos nulos
cancer.isnull()
cancer.drop(columns=["Unnamed: 32","id"], inplace = True)
# el segundo cambio es de la columna 1 "diagnosis" cambiar el tipo abject de valores "M" para maligno y "B" para benigno 
# a 0 y 1
cancer['diagnosis'] = cancer['diagnosis'].map({'M':1, 'B':0})
print(cancer.info())

Hemos eliminado la columna "id" que no nos aportaba nada, la columna "Unnamed: 32" que eran todo valores nulos y, por último, hemos transformado los datos de la columna "diagnosis" porque no eran de tipo numerico y no nos servirian para entrenar al modelo correctamente. 

In [ ]:
print(cancer.duplicated().sum()) #duplicados

Ahora que ya estan limpios nuestros datos vamos a visualizarlos antes de entrenar a nuestro modelo

In [ ]:
# vamos a ver que variables estan correlacionadas con la columna objetivo 
cancer.corr()['diagnosis'].sort_values(ascending=False)

In [ ]:
# ahora vamos a obtener las características con mayor varianza, que suelen tener informacion valiosa
cancer.var().sort_values(ascending=False)

Los graficos con puntos de distintos colores bien separados indica que las variables de esa grafica podrian ser muy relevantes para el modelo.

En cambio cuando vemos superposicion de colores, esas variables suelen tener poco valor predictivo po si solas.

Como usar pairplot nos daba demasiados datos vamos a usar las variables con alta correlacion con la columna "diagnosis" y las variables con mayor varianza, que suelen tener mas informacion.

criterios para seleccionar las variables:
- correlacion >= 0.5 (muestra una fuerte relacion con el diagnostico)
- varianza >= 1000 (evitara incluir variables con cambios insignificantes)

Variables con alta correlacion y buena varianza:
- concave points_worst (corr = 0.79, var = 0.004)

- perimeter_worst (corr = 0.78, var = 1129)

- concave points_mean (corr = 0.77, var = 0.001)

- radius_worst (corr = 0.77, var = 23)

- perimeter_mean (corr = 0.74, var = 590)

- area_worst (corr = 0.73, var = 324167)

- radius_mean (corr = 0.73, var = 12)

- area_mean (corr = 0.70, var = 123843)

In [ ]:
# usaremos un grafico que nos muestre como se relacionan estas variables con el objetivo "diagnosis"

cols = [
    'diagnosis', 'concave points_worst', 'perimeter_worst', 'concave points_mean', 
    'radius_worst', 'perimeter_mean', 'area_worst', 'radius_mean', 'area_mean'
]
sns.pairplot(cancer[cols], hue='diagnosis')

In [ ]:
# vamos a probar con un mapa de calor para detectar patrones
plt.figure(figsize=(8,10))
sns.heatmap(cancer[cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de correlación')
plt.show()

A continuación se muestran tres graficas con variables altamente correlacionadas

In [ ]:
sns.scatterplot(data=cancer, x='perimeter_worst', y='area_worst', hue='diagnosis')

In [ ]:
sns.scatterplot(data=cancer, x='perimeter_worst', y='radius_worst', hue='diagnosis')

In [ ]:
sns.scatterplot(data=cancer, x='perimeter_mean', y='radius_mean', hue='diagnosis')

## **División del Dataset**
Se divide el conjunto de datos en entrenamiento (80%) y prueba (20%).

Esto nos va a permitir evaluar el rendimiento real de los modelos con datos no vistos.

In [ ]:
X = cancer.drop('diagnosis', axis=1)
y = cancer['diagnosis']

# división inicial
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **Escalado de Datos**
Como vamos a usar Regresión Logística y KNN, que son modelos sensibles a la escala de datos, es imprescindible normalizar los datos antes de usarlos.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # el conjunto de prueba se transforma usando el mismo scaler

## **Validación Cruzada con K-Fold (k=5 y k=10)**

Cada modelo se evaluará usando 5-fold y 10-fold para comparar resultados.

Aplicamos un modelo de Árbol de Decision usando validación cruzada k-fold con distintos valores de k (k=5, k=10)

In [ ]:
# Modelo de arbol de decisión
tree_model = DecisionTreeClassifier(random_state=42)

# Validación cruzada con k=5
scores_5 = cross_val_score(tree_model, X_train_scaled, y_train, cv=5, scoring='accuracy')

# Validación cruzada con  k=10
scores_10 = cross_val_score(tree_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

print(f"Precisión Árbol (k=5): {scores_5.mean():.4f}")
print(f"Precisión Árbol (k=10): {scores_10.mean():.4f}")

Aplicamos otros modelos: Regresión Logística y K-NN y repetir la Validación Cruzada.

In [ ]:
# Modelo de Regresión Logística
logreg_model = LogisticRegression(max_iter=10000, random_state=42)

scores_logreg_5 = cross_val_score(logreg_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
scores_logreg_10 = cross_val_score(logreg_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

print(f"Precisión Logística (k=5): {scores_logreg_5.mean():.4f}")
print(f"Precisión Logística (k=10): {scores_logreg_10.mean():.4f}")

In [ ]:
# Modelo KNN con k=5
knn_model = KNeighborsClassifier(n_neighbors=5)

scores_knn_5 = cross_val_score(knn_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
scores_knn_10 = cross_val_score(knn_model, X_train_scaled, y_train, cv=10, scoring='accuracy')

print(f"Precisión KNN (k=5): {scores_knn_5.mean():.4f}")
print(f"Precisión KNN (k=10): {scores_knn_10.mean():.4f}")

## **Comparación de Resultados de la Validación Cruzada**

Creamos un DataFrame con las precisiones medias para facilitar la visualización de los resultados.

In [ ]:
resultados = pd.DataFrame({
    'Modelo': ['Árbol de Decisión', 'Regresión Logística', 'KNN'],
    'Precisión k=5': [scores_5.mean(), scores_logreg_5.mean(), scores_knn_5.mean()],
    'Precisión k=10': [scores_10.mean(), scores_logreg_10.mean(), scores_knn_10.mean()]
})

print(resultados)

## **Análisis del Impacto del Valor de K**

Primero se va a mostrar los gráficos individuales para analizar el impacto de k y despues habrá un gráfico comparativo de los modelos que se han usado.

In [ ]:
#Arbol de decision
k_values = range(3, 16)  # Valores de k de 3 a 15
tree_scores = [cross_val_score(tree_model, X_train_scaled, y_train, cv=k, scoring='accuracy').mean() for k in k_values]

plt.plot(k_values, tree_scores, marker='o', label='Árbol de Decisión')
plt.xlabel('Número de Folds (k)')
plt.ylabel('Precisión Media')
plt.title('Rendimiento del Árbol de Decisión')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#Regresión Logística
logreg_scores = [cross_val_score(logreg_model, X_train_scaled, y_train, cv=k, scoring='accuracy').mean() for k in k_values]

plt.plot(k_values, logreg_scores, marker='o', label='Regresión Logística')
plt.xlabel('Número de Folds (k)')
plt.ylabel('Precisión Media')
plt.title('Rendimiento de Regresión Logística')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# KNN
knn_scores = [cross_val_score(knn_model, X_train_scaled, y_train, cv=k, scoring='accuracy').mean() for k in k_values]

plt.plot(k_values, knn_scores, marker='o', label='KNN')
plt.xlabel('Número de Folds (k)')
plt.ylabel('Precisión Media')
plt.title('Rendimiento de KNN')
plt.grid(True)
plt.legend()
plt.show()

## **Gráfico Comparativo de Modelos**

Con esta gráfica vamos a poder comparar visualmente las precisiones alcanzadas por cada modelo con k=5 y k=10.

In [ ]:
# gráfico comparativo

plt.figure(figsize=(8, 5))
resultados_melted = resultados.melt(id_vars='Modelo', 
                                    value_vars=['Precisión k=5', 'Precisión k=10'],
                                    var_name='k',
                                    value_name='Precisión')


sns.barplot(x='Modelo', y='Precisión', hue='k', data=resultados_melted)

plt.title('Comparación de precisión entre modelos (k=5 vs k=10)')
plt.ylabel('Precisión Media')
plt.legend(title='Valor de k')
plt.show()

## **Conclusiones**

- **Comparación entre Modelos :** El modelo de Regresión Logística ha mostrado un rendimiento más consistente en los dos valores de k que hemos probado, esto nos sugiere que este modelos puede generalizar mejor en este conjunto de datos. El modelo de Árbol de Decisión es el que ha variado más cuando cambiamos el valor de k. 

- **Impacto del Valor de K :** El uso de k=10 nos ha proporcionado resultados más estables que cuando hemos usado k=5, sobretodo en modelos como KNN y Árboles de Decisión. Esto es porqué un mayor número de folds reduce el sesgo pero incrementa la varianza en el entrenamiento.

- **Modelo más Eficaz :** En este caso, la Regresión Logística ha sido la más precisa y estable, así que, esta sería la elección más adecuada para el caso de este conjunto de datos.
